# Bodybuilding WFPB Diet
Author: Simon Thornewill von Essen

Date: 2019-05-02

## Overview

In this jupyter notebook I want to do analysis on the data that I have collected on various foods that I buy on a daily basis and see what the optimal mix of these foods are to meet my daily requirements for bodybuilding. (For more information, see the README.)

I want to do this analysis in a number of steps:

1. Import relevant data and packages
2. Quickly clean the data so that it is ready for analysis
    * Should be relatively quick because I had to create this data by hand so it is mostly in the format I want already.
3. Do a quick exploratory data analysis to get some understanding of the nutrients in my food
4. Use linear programming to determine the optimal mix of foods while minimising the total cost

All of the values used for nutrients are from cronometer.com and numbers for EDAs and other such targets can be found in the README.

## Step 1 Import Data and packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

plt.style.use("seaborn")

%matplotlib inline

In [2]:
df = pd.read_csv("./dat/foods.csv")

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,NaN,NaN,100.0,0.14,45.0,NaN,NaN,85.0,NaN,...,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN
1,soy_mlk,NaN,NaN,100.0,0.14,46.0,NaN,NaN,85.0,NaN,...,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN
2,lentils,NaN,100.0,NaN,0.72,358.0,NaN,NaN,7.8,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,NaN,NaN,3.6
3,banana,NaN,100.0,NaN,0.20,89.0,NaN,NaN,74.9,22.8,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,NaN,100.0,NaN,0.16,32.0,NaN,NaN,91.0,7.7,...,16.0,NaN,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


## Clean Data
You can see above a list of foods, cost, quantity, and their associated nutritional information. 

One thing I did deliberately while building this dataset by hand was leaving Null values. This is because I intended to fill them with `0`s later using pandas's functionality to do such things.

In [3]:
df.fillna(value=0, inplace=True)

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,alcohol_g,caffeine_mg,water_g,carbs_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,0.0,0.0,100.0,0.14,45.0,0.0,0.0,85.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,0.0,0.0,100.0,0.14,46.0,0.0,0.0,85.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,0.0,100.0,0.0,0.72,358.0,0.0,0.0,7.8,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,0.0,100.0,0.0,0.20,89.0,0.0,0.0,74.9,22.8,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,0.0,100.0,0.0,0.16,32.0,0.0,0.0,91.0,7.7,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


Now that the null values have been filled, I want to drop columns that have only `0` values in them. Columns like alcohol and caffiene have been completely unused for example.

In [4]:
# Find columns that have a sum of 0, i.e. completely unused
df.sum() 

food_item            oat_soy_mlksoy_mlklentilsbananastrawberrypeanu...
pcs                                                                  2
pcs_g                                                             2546
pcs_ml                                                             200
cost_eur                                                         13.23
energy_kcal                                                     4630.8
alcohol_g                                                            0
caffeine_mg                                                          0
water_g                                                         1827.5
carbs_g                                                          460.4
fiber_g                                                          178.2
starch_g                                                         142.3
sugars_g                                                         113.8
fat_g                                                            251.4
monoun

In [5]:
# Find only food with vitamin D
df.query("vit_d_IU > 0").food_item

14    champignons
Name: food_item, dtype: object

We can see in the columns above that the only other unused column is cholesterol, and so I'll drop that as well as the other two.

The only other two columns are vitamin B-12, which I am not concerned about since I take a suppliment and vitamin-D, which can only be found in mushrooms.

So I'll drop these columns

In [6]:
df.drop(labels=["cholesterol_mg", "vit_d_IU", "alcohol_g", "caffeine_mg"],
       axis=1,
       inplace = True)

df.head()

,food_item,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,water_g,carbs_g,fiber_g,starch_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,0.0,0.0,100.0,0.14,45.0,85.0,0.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,0.0,0.0,100.0,0.14,46.0,85.0,0.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,0.0,100.0,0.0,0.72,358.0,7.8,63.1,63.1,47.6,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,0.0,100.0,0.0,0.20,89.0,74.9,22.8,2.6,5.4,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,0.0,100.0,0.0,0.16,32.0,91.0,7.7,2.0,0.0,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


I suppose the only difficulty left is that when doing the linear programming that there is no one clear column that can be used to tweak the amounts of values. Instead, we have three columns that can be used towards this purpose, `pcs`, `pcs_g` and `pcs_ml`.

these should be combined into a single column for use in linear programming.

In [7]:
# Create dataframe
df_amount = df[["pcs", "pcs_g", "pcs_ml"]]

df_amount.head()

,pcs,pcs_g,pcs_ml
0,0.0,0.0,100.0
1,0.0,0.0,100.0
2,0.0,100.0,0.0
3,0.0,100.0,0.0
4,0.0,100.0,0.0


In [8]:
# Replace 0 values with NaN
df_amount.replace(to_replace=0,
                 value=np.NaN,
                 inplace = True);

# Coalesce columns
df["amount"] =  df_amount["pcs_g"].combine_first(df_amount["pcs_ml"].combine_first(df_amount["pcs"]))

# Reordering columns
df = df[['food_item','amount', 'pcs', 'pcs_g', 'pcs_ml', 'cost_eur', 'energy_kcal',
       'water_g', 'carbs_g', 'fiber_g', 'starch_g', 'sugars_g', 'fat_g',
       'monounsaturated_g', 'polyunsaturated_g', 'omega_3_g', 'omega_6_g',
       'saturated_g', 'trans_fats_g', 'protein_g', 'vit_b1_mg', 'vit_b2_mg',
       'vit_b3_mg', 'vit_b5_mg', 'vit_b6_mg', 'vit_b12_ug', 'folate_ug',
       'vit_a_IU', 'vit_c_mg', 'vit_e_mg', 'vit_k_ug', 'calcium_mg',
       'copper_mg', 'iron_mg', 'magnesium_mg', 'manganese_mg', 'phosphorus_mg',
       'potassium_mg', 'selenium_ug', 'sodium_mg', 'zinc_mg']]

df.head()

C:\Users\sthornewillvonessen\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,food_item,amount,pcs,pcs_g,pcs_ml,cost_eur,energy_kcal,water_g,carbs_g,fiber_g,...,calcium_mg,copper_mg,iron_mg,magnesium_mg,manganese_mg,phosphorus_mg,potassium_mg,selenium_ug,sodium_mg,zinc_mg
0,oat_soy_mlk,100.0,0.0,0.0,100.0,0.14,45.0,85.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
1,soy_mlk,100.0,0.0,0.0,100.0,0.14,46.0,85.0,0.0,0.0,...,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0
2,lentils,100.0,0.0,100.0,0.0,0.72,358.0,7.8,63.1,63.1,...,48.0,1.3,7.4,5.9,1.7,294.0,668.0,0.0,0.0,3.6
3,banana,100.0,0.0,100.0,0.0,0.20,89.0,74.9,22.8,2.6,...,5.0,0.1,0.3,27.0,0.3,22.0,358.0,1.0,1.0,0.2
4,strawberry,100.0,0.0,100.0,0.0,0.16,32.0,91.0,7.7,2.0,...,16.0,0.0,0.4,13.0,0.4,24.0,153.0,0.4,1.0,0.1


## Exploratory Data Analysis